In [58]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv
import datetime


# create Pandas dataframe to store the data
talks_columns = ['start','end', 'Speaker(s)','Title','Title_text','url', 'location']
talks = pd.DataFrame(columns = talks_columns)

# bcs mit events 
row = 0
for i in range(3):
    print(i)
    page = requests.get("https://bcs.mit.edu/news-events/events?page="+str(i))
    soup = BeautifulSoup(page.content,'html.parser')
    TalkinfoAll = soup.select("div.node-content")
    for j in range(len(TalkinfoAll)):
        Talkinfo = TalkinfoAll[j]
        
        # find the title and url
        title_tag = Talkinfo.select("h3.node-title")[0].a
        talks.loc[row, 'Title_text'] = title_tag.get_text()
        talks.loc[row, 'url'] = 'https://bcs.mit.edu'+ title_tag['href']
        title = str(title_tag)
        talks.loc[row, 'Title'] = title[0:9]+'https://bcs.mit.edu'+title[9:]
        
        # find the locations 
        subpage = requests.get(talks.loc[row, 'url'])
        subsoup = BeautifulSoup(subpage.content,'html.parser')
        talks.loc[row,'location'] = subsoup.select("var.atc_location")[0].get_text()
        
        # find the speaker info, and speaker url if exists. 
        speaker_tag = Talkinfo.select("div.textformatter-list")
        if len(speaker_tag) != 0:
            talks.loc[row, 'Speaker(s)'] = speaker_tag[0].get_text()
        else:
            talks.loc[row, 'Speaker(s)'] = 'TBA'

            
        # find the datatime and seperate them     
        talks.loc[row, 'start'] = str(datetime.datetime.strptime(Talkinfo.select("var.atc_date_start")[0].get_text(),'%Y/%m/%d %I:%M:%S %p').strftime('%Y-%m-%dT%H:%M:%S'))
        talks.loc[row, 'end'] = str(datetime.datetime.strptime(Talkinfo.select("var.atc_date_end")[0].get_text(),'%Y/%m/%d %I:%M:%S %p').strftime('%Y-%m-%dT%H:%M:%S'))
        row = row + 1

0
1
2


In [59]:
def replace_noon(datetime):
    datetime = datetime.split(" ")
    time = datetime[3]
    if time == 'noon': 
        time = '12:00pm'
        
    return datetime[0]+' '+datetime[1]+' '+datetime[2]+' '+time

# harvard events 
page = requests.get("http://cbs.fas.harvard.edu/resources/events")
soup = BeautifulSoup(page.content,'html.parser')
TalkinfoAll = soup.select("div.view-content")[1]
TalkinfoAll = TalkinfoAll.find_all("div", recursive=False)
for j in range(len(TalkinfoAll)):
        Talkinfo = TalkinfoAll[j]
        
        # find the title and url
        title_tag = Talkinfo.select("h3.featuredtext")[0].a
        talks.loc[row, 'Title_text'] = title_tag.get_text()
        talks.loc[row, 'url'] = 'http://cbs.fas.harvard.edu'+ title_tag['href']
        title = str(title_tag)
        talks.loc[row, 'Title'] = title[0:9]+'http://cbs.fas.harvard.edu/'+title[9:]
        
        # find the locations 
        talks.loc[row,'location'] = list(list(Talkinfo.children)[5].children)[2]
        
        # find the speaker info, and speaker url if exists. 
        talks.loc[row, 'Speaker(s)'] = list(Talkinfo.children)[4]
            
        # find the datatime and seperate them     
        tmp = replace_noon(Talkinfo.select("span.date-display-single")[0].get_text())
        tmp = '2018 '+ tmp
        talks.loc[row, 'start'] = str(datetime.datetime.strptime(tmp, '%Y %a %d %b %I:%M%p').strftime('%Y-%m-%dT%H:%M:%S')) 
        row = row + 1

talks = talks.replace('\n','', regex=True)
talks.end.fillna(talks.start, inplace=True)


In [60]:
for i in range(len(talks)): 
    event = {
   'summary': talks.loc[i, 'Speaker(s)'] +': '+talks.loc[i, 'Title_text'],
   'location': talks.loc[i, 'location'],
   'description': 'For details: link here: '+ talks.loc[i, 'url'],
   'start': {
     'dateTime': talks.loc[i, 'start'],
     'timeZone': 'America/New_York',
   },
   'end': {
     'dateTime': talks.loc[i, 'end'],
     'timeZone': 'America/New_York',
   },
   }
    print('*'*100)
    print(event)

****************************************************************************************************
{'summary': 'Julian Jara-Ettinger, Ph.D.: SCSB Lunch Series - Julian Jara-Ettinger: Computational principles underlying social cognition: similarities and differences in ASD', 'location': 'Simons Center Conference room 46-6011', 'description': 'For details: link here: https://bcs.mit.edu/news-events/events/scsb-lunch-series-julian-jara-ettinger-computational-principles-underlying-social', 'start': {'dateTime': '2018-04-27T12:00:00', 'timeZone': 'America/New_York'}, 'end': {'dateTime': '2018-04-27T13:00:00', 'timeZone': 'America/New_York'}}
****************************************************************************************************
{'summary': "Julia Leonard, Schulz/Gabrieli Lab: Social Influences on children's learning", 'location': 'Brain and Cognitive Sciences Complex, 43 Vassar Street, Singleton Auditorium and 3rd floor Atrium, Cambridge MA', 'description': 'For details: link

In [55]:
talks

,start,end,Speaker(s),Title,Title_text,url,location
0,2018-04-27T12:00:00,2018-04-27T13:00:00,"Julian Jara-Ettinger, Ph.D.","<a href=""https://bcs.mit.edu/news-events/event...",SCSB Lunch Series - Julian Jara-Ettinger: Comp...,https://bcs.mit.edu/news-events/events/scsb-lu...,Simons Center Conference room 46-6011
1,2018-04-27T14:00:00,2018-04-27T16:00:00,"Julia Leonard, Schulz/Gabrieli Lab","<a href=""https://bcs.mit.edu/news-events/event...",Social Influences on children's learning,https://bcs.mit.edu/news-events/events/tba-43,"Brain and Cognitive Sciences Complex, 43 Vassa..."
2,2018-04-30T10:00:00,2018-04-30T12:00:00,"<a href=""https://scholar.harvard.edu/rachelrom...","<a href=""https://bcs.mit.edu/news-events/event...",Environmental influences on the neural basis o...,https://bcs.mit.edu/news-events/events/environ...,Singleton Auditorium 46-3002
3,2018-05-01T11:00:00,2018-05-01T12:00:00,"<a href=""http://www.neurorepair.net/"">James Bo...","<a href=""https://bcs.mit.edu/news-events/event...",The marmoset monkey as a model for studying br...,https://bcs.mit.edu/news-events/events/marmose...,"McGovern Seminar Room, 46-3189"
4,2018-05-03T16:00:00,2018-05-03T17:00:00,TBA,"<a href=""https://bcs.mit.edu/news-events/event...",Colloquium - Marlene Cohen,https://bcs.mit.edu/news-events/events/colloqu...,Singleton Auditorium 46-3002
5,2018-05-04T12:00:00,2018-05-04T13:00:00,"Sasha Krol, Ph.D., Simons Fellow","<a href=""https://bcs.mit.edu/news-events/event...","SCSB Lunch Series - Sasha Krol, Ph.D.: ""Uncove...",https://bcs.mit.edu/news-events/events/scsb-lu...,Simons Center Conference room 46-6011
6,2018-05-04T14:00:00,2018-05-04T15:00:00,"Jon Shlens, Google Brain","<a href=""https://bcs.mit.edu/news-events/event...","Brains, Minds + Machines Seminar Series: Learn...",https://bcs.mit.edu/news-events/events/brains-...,Singleton Auditorium (46-3002) - 43 Vassar Str...
7,2018-05-04T16:00:00,2018-05-04T18:00:00,"<a href=""http://www.childrenshospital.org/rese...","<a href=""https://bcs.mit.edu/news-events/event...",The Molecular & Cellular Neuroscience (MCN) Pr...,https://bcs.mit.edu/news-events/events/molecul...,46-3002 Singleton Auditorium
8,2018-05-09T09:00:00,2018-05-09T17:00:00,TBA,"<a href=""https://bcs.mit.edu/news-events/event...",Picower Institute Spring 2018 Symposium: Early...,https://bcs.mit.edu/news-events/events/picower...,"Singleton Auditorium, 46-3002"
9,2018-05-10T16:00:00,2018-05-10T17:00:00,Daniel Johnston,"<a href=""https://bcs.mit.edu/news-events/event...",(Canceled) Colloquium - Daniel Johnston,https://bcs.mit.edu/news-events/events/cancele...,Singleton Auditorium 46-3002
